In [1]:
import os
import sys
import shutil
import argparse
import traceback
import csv
import pandas as pd
import datetime as dt
import numpy as np
from pprint import pprint
from tqdm import tqdm
from pytictoc import TicToc

# ******************************* User Settings *******************************
class Payload:
    LENGTH = 250              # (Bytes)
    TAG = "000425d401df5e76"  # 2 71828 3 1415926 (hex)            : 8-bytes
    OFS_TIME = (16, 24)       # epoch time of 'yyyy/mm/dd hh:mm:ss': 4-bytes
    OFS_USEC = (24, 32)       # microsecond (usec)                 : 4-bytes
    OFS_SEQN = (32, 40)       # sequence number (start from 1)     : 4-bytes
class ServerIP:
    PUBLIC = "140.112.20.183"  # 2F    
    PRIVATE = "192.168.1.248"  # 2F
    # PUBLIC = "140.112.17.209"  # 3F
    # PRIVATE = "192.168.1.108"  # 3F
# *****************************************************************************

### Packet Loss Aggregate

In [2]:
# ******************************* User Settings *******************************
database = "/home/wmnlab/D/database/"
date = "2022-10-11"
db_path = os.path.join(database, date)
Exp_Name = {  # experiment_name:(number_of_experiment_rounds, list_of_experiment_round)
                # If the list is empty, it will list all directories in the current directory by default.
                # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "_Bandlock_Udp":(1, ["#01"]),
    # "_Bandlock_Udp":(5, ["#02", "#03", "#04", "#05", "#06"]),
    # "_Bandlock_Udp":(4, ["#01", "#02", "#03", "#04"]),
    # "_Bandlock_Udp":(6, []),
    "_Bandlock_Udp":(4, []),
    # "_Udp_Stationary_Bandlock":(1, []), 
    # "_Udp_Stationary_SameSetting":(1, []),
}
devices = [
    "sm03",
    "sm04",
    "sm05", 
    "sm06",
    "sm07",
    "sm08",
]
# settings = [
#     "twm_all",
#     "twm_b1",
#     "twm_b3",
#     "twm_b28"
# ]
settings = [
    "cht_all",
    "twm_all_1",
    "twm_all_2",
    "twm_b1b3",
    "twm_b3b28",
    "twm_b1b28"
]
# settings = [
#     "twm_all_1",
#     "twm_all_2",
#     "twm_gsm",
#     "twm_all_4",
#     "twm_all_5"
# ]
# *****************************************************************************

for _exp, (_times, _rounds) in Exp_Name.items():
    ### Check if these directories exist
    exp_path = os.path.join(db_path, _exp)
    print(exp_path)
    exp_dirs = []
    for i, dev in enumerate(devices):
        if _rounds:
            exp_dirs.append([os.path.join(exp_path, dev, _round) for _round in _rounds])
        else:
            _rounds = sorted(os.listdir(os.path.join(exp_path, dev)))
            exp_dirs.append([os.path.join(exp_path, dev, item) for item in _rounds])
        exp_dirs[i] = [item for item in exp_dirs[i] if os.path.isdir(item)]
        print(_times)
        pprint(exp_dirs[i])
        if len(exp_dirs[i]) != _times:
            print("************************************************************************************************")
            print("Warning: the number of directories does not match your specific number of experiment times.")
            print("************************************************************************************************")
            print()
            sys.exit()
    print()

    dfs_dl = {}
    hos_dl = {}
    dfs_ul = {}
    hos_ul = {}
    for dev in devices:
        dfs_dl[dev] = []
        hos_dl[dev] = []
        dfs_ul[dev] = []
        hos_ul[dev] = []

    ### Read files
    print(_exp)
    for j in range(_times):
        for i, dev in enumerate(devices):
            print(exp_dirs[i][j])
            dirpath = os.path.join(exp_dirs[i][j], "analysis")
            dfs_dl[dev].append(pd.read_csv(os.path.join(dirpath, "dwnlnk_udp_loss_statistics.csv")))
            # hos_dl[dev].append(pd.read_csv(os.path.join(dirpath, "class-ratio-50p", "dwnlnk_ho_statistics.csv")))
            hos_dl[dev].append(pd.read_csv(os.path.join(dirpath, "udp-loss-classify-50p", "dwnlnk_ho_statistics.csv")))
            dfs_ul[dev].append(pd.read_csv(os.path.join(dirpath, "uplnk_udp_loss_statistics.csv")))
            # hos_ul[dev].append(pd.read_csv(os.path.join(dirpath, "class-ratio-50p", "uplnk_ho_statistics.csv")))
            hos_ul[dev].append(pd.read_csv(os.path.join(dirpath, "udp-loss-classify-50p", "uplnk_ho_statistics.csv")))
            # print(dfs_dl[dev])
            # print(dfs_ul[dev])
        print()

    print("**********************************************************************")
    for dev, setting in zip(devices, settings):
        for i in range(_times):
            dfs_dl[dev][i] = pd.concat([dfs_dl[dev][i], hos_dl[dev][i][['succ_handoff', 'fail_handoff', 'overall_handoff']]], axis=1)
            dfs_dl[dev][i] = pd.concat([dfs_dl[dev][i], hos_dl[dev][i]["lte_handover,SN_addition,SN_removal,endc_SN_change,endc_MN_change,endc_MNSN_change,lte2endc_MN_change,endc2lte_MN_change,scg_failure,radio_link_failure,nas_recovery".split(',')]], axis=1)
            # dfs_dl[dev][i].rename(columns={'overall':'handoff_num'}, inplace=True)
            dfs_ul[dev][i] = pd.concat([dfs_ul[dev][i], hos_ul[dev][i][['succ_handoff', 'fail_handoff', 'overall_handoff']]], axis=1)
            dfs_ul[dev][i] = pd.concat([dfs_ul[dev][i], hos_ul[dev][i]["lte_handover,SN_addition,SN_removal,endc_SN_change,endc_MN_change,endc_MNSN_change,lte2endc_MN_change,endc2lte_MN_change,scg_failure,radio_link_failure,nas_recovery".split(',')]], axis=1)
            # dfs_ul[dev][i].rename(columns={'overall':'handoff_num'}, inplace=True)
    
    for dev, setting in zip(devices, settings):
        # for i in range(1, _times):
            # dfs_dl[dev][0].append(dfs_dl[dev][i])
            # dfs_ul[dev][0].append(dfs_ul[dev][i])
            ### FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
        dfs_dl[dev][0] = pd.concat(dfs_dl[dev])
        dfs_ul[dev][0] = pd.concat(dfs_ul[dev])
        dfs_dl[dev][0].reset_index(drop=True, inplace=True)
        dfs_ul[dev][0].reset_index(drop=True, inplace=True)
        # dfs_dl[dev][0] = dfs_dl[dev][0].reindex(['id.'] + dfs_dl[dev][0].columns.tolist(), axis=1)
        # dfs_ul[dev][0] = dfs_ul[dev][0].reindex(['id.'] + dfs_ul[dev][0].columns.tolist(), axis=1)
        # for i, s in enumerate(_rounds):
        #     dfs_dl[dev][0].loc[i, 'id.'] = s
        #     dfs_ul[dev][0].loc[i, 'id.'] = s
        # dfs_dl[dev][0].loc['total'] = dfs_dl[dev][0].sum()
        # dfs_ul[dev][0].loc['total'] = dfs_ul[dev][0].sum()
        dfs_dl[dev][0].loc["overall"] = dfs_dl[dev][0].sum()
        dfs_ul[dev][0].loc["overall"] = dfs_ul[dev][0].sum()
        # print("**************************************************")
        # dfs_dl[dev][0].loc['total', 'packet_loss_rate(%)'] = dfs_dl[dev][0]['total_packet_loss'].iloc[:-1].sum() / dfs_dl[dev][0]['total_packet_sent'].iloc[:-1].sum() * 100
        # dfs_ul[dev][0].loc['total', 'packet_loss_rate(%)'] = dfs_ul[dev][0]['total_packet_loss'].iloc[:-1].sum() / dfs_ul[dev][0]['total_packet_sent'].iloc[:-1].sum() * 100
        # dfs_dl[dev][0].loc['total', 'packet_loss_rate(%)'] = dfs_dl[dev][0].loc['total', 'total_packet_loss'] / dfs_dl[dev][0].loc['total', 'total_packet_sent'] * 100
        # dfs_ul[dev][0].loc['total', 'packet_loss_rate(%)'] = dfs_ul[dev][0].loc['total', 'total_packet_loss'] / dfs_ul[dev][0].loc['total', 'total_packet_sent'] * 100
        dfs_dl[dev][0].loc["overall", 'packet_loss_rate(%)'] = dfs_dl[dev][0].loc["overall", 'total_packet_loss'] / (dfs_dl[dev][0].loc["overall", 'total_packet_sent'] + 1e-9) * 100
        dfs_ul[dev][0].loc["overall", 'packet_loss_rate(%)'] = dfs_ul[dev][0].loc["overall", 'total_packet_loss'] / (dfs_ul[dev][0].loc["overall", 'total_packet_sent'] + 1e-9) * 100
        dfs_dl[dev][0].to_csv(os.path.join(exp_path, dev, "dwnlnk_udp_loss_statistics_meta_{}.csv".format(setting)), index=True, index_label='type')
        print(os.path.join(exp_path, dev, "dwnlnk_udp_loss_statistics_meta_{}.csv".format(setting)))
        dfs_ul[dev][0].to_csv(os.path.join(exp_path, dev, "uplnk_udp_loss_statistics_meta_{}.csv".format(setting)), index=True, index_label='type')
        print(os.path.join(exp_path, dev, "uplnk_udp_loss_statistics_meta_{}.csv".format(setting)))

/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#03',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#04']
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#03',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#04']
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#03',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#04']
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#03',
 '/home/wmnla

### Packet Loss Classify Aggregate

In [3]:
# ******************************* User Settings *******************************
database = "/home/wmnlab/D/database/"
date = "2022-10-11"
db_path = os.path.join(database, date)
Exp_Name = {  # experiment_name:(number_of_experiment_rounds, list_of_experiment_round)
                # If the list is empty, it will list all directories in the current directory by default.
                # If the number of experiment times != the length of existing directories of list, it would trigger warning and skip the directory.
    # "_Bandlock_Udp":(1, ["#01"]),
    # "_Bandlock_Udp":(5, ["#02", "#03", "#04", "#05", "#06"]),
    # "_Bandlock_Udp":(4, ["#01", "#02", "#03", "#04"]),
    # "_Bandlock_Udp":(6, []),
    "_Bandlock_Udp":(4, []),
    # "_Udp_Stationary_Bandlock":(1, []), 
    # "_Udp_Stationary_SameSetting":(1, []),
}
devices = [
    "sm03",
    "sm04",
    "sm05", 
    "sm06",
    "sm07",
    "sm08",
]
# settings = [
#     "twm_all",
#     "twm_b1",
#     "twm_b3",
#     "twm_b28"
# ]
settings = [
    "cht_all",
    "twm_all_1",
    "twm_all_2",
    "twm_b1b3",
    "twm_b3b28",
    "twm_b1b28"
]
# settings = [
#     "twm_all_1",
#     "twm_all_2",
#     "twm_gsm",
#     "twm_all_4",
#     "twm_all_5"
# ]
# *****************************************************************************

ho_types = "lte_handover,SN_addition,SN_removal,endc_SN_change,endc_MN_change,endc_MNSN_change,lte2endc_MN_change,endc2lte_MN_change".split(',')
rlf_types = "scg_failure,radio_link_failure,nas_recovery".split(',')

for _exp, (_times, _rounds) in Exp_Name.items():
    ### Check if these directories exist
    exp_path = os.path.join(db_path, _exp)
    print(exp_path)
    exp_dirs = []
    for i, dev in enumerate(devices):
        if _rounds:
            exp_dirs.append([os.path.join(exp_path, dev, _round) for _round in _rounds])
        else:
            _rounds = sorted(os.listdir(os.path.join(exp_path, dev)))
            exp_dirs.append([os.path.join(exp_path, dev, item) for item in _rounds])
        exp_dirs[i] = [item for item in exp_dirs[i] if os.path.isdir(item)]
        print(_times)
        pprint(exp_dirs[i])
        if len(exp_dirs[i]) != _times:
            print("************************************************************************************************")
            print("Warning: the number of directories does not match your specific number of experiment times.")
            print("************************************************************************************************")
            print()
            sys.exit()
    print()

    dfs_dl = {}
    dfs_ul = {}
    for dev in devices:
        dfs_dl[dev] = []
        dfs_ul[dev] = []

    ### Read files
    print(_exp)
    for j in range(_times):
        for dev in devices:
            dfs_dl[dev] = []
            dfs_ul[dev] = []
        for i, dev in enumerate(devices):
            print(exp_dirs[i][j])
            dirpath = os.path.join(exp_dirs[i][j], "analysis", "udp-loss-classify-50p")
            for k in range(10):
                dfs_dl[dev].append(pd.read_csv(os.path.join(dirpath, "dwnlnk_loss_ho_classify_0{}.csv".format(k))))
                dfs_ul[dev].append(pd.read_csv(os.path.join(dirpath, "uplnk_loss_ho_classify_0{}.csv".format(k))))
            for k in range(1, 11):
                dfs_dl[dev].append(pd.read_csv(os.path.join(dirpath, "dwnlnk_loss_ho_classify_{}.csv".format(k))))
                dfs_ul[dev].append(pd.read_csv(os.path.join(dirpath, "uplnk_loss_ho_classify_{}.csv".format(k))))
            dl_df = dfs_dl[dev][0].set_index('type').transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index' : 'metrics'})
            ul_df = dfs_ul[dev][0].set_index('type').transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index' : 'metrics'})
            for k in range(1, len(dfs_dl[dev])):
                dl_df = pd.concat([dl_df, dfs_dl[dev][k].set_index('type').transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index' : 'metrics'})])
                ul_df = pd.concat([ul_df, dfs_ul[dev][k].set_index('type').transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index' : 'metrics'})])
            dl_df = dl_df.reindex(['window_size'] + dl_df.columns.tolist(), axis=1)
            ul_df = ul_df.reindex(['window_size'] + ul_df.columns.tolist(), axis=1)
            dl_df.reset_index(drop=True, inplace=True)
            ul_df.reset_index(drop=True, inplace=True)
            ss = [i/10 for i in range(10)] + list(range(1, 11))
            for i, val in enumerate(ss):
                dl_df.loc[5*i:5*(i+1), 'window_size'] = val
                ul_df.loc[5*i:5*(i+1), 'window_size'] = val
            dl_df.to_csv(os.path.join(dirpath, "dwnlnk_loss_ho_classify_total.csv".format(setting)), index=True)
            ul_df.to_csv(os.path.join(dirpath, "uplnk_loss_ho_classify_total.csv".format(setting)), index=True)
            # print(dfs_dl[dev])
            # print(dfs_ul[dev])
        print()

/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#03',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm03/#04']
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#03',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#04']
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#03',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm05/#04']
4
['/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#01',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#02',
 '/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm06/#03',
 '/home/wmnla

In [31]:
# df = pd.read_csv("/home/wmnlab/D/database/2022-10-11/_Bandlock_Udp/sm04/#02/analysis/udp-loss-classify-50p/dwnlnk_loss_ho_classify_1.csv")
# # print(df)
# df = df.set_index('type').transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index' : 'metrics'})
# df = df.reindex(['window_size'] + df.columns.tolist(), axis=1)
# df.loc[0:3, 'window_size'] = 0
# df.loc[3:5, 'window_size'] = 1
# print(df)

   window_size              metrics before_lte_handover during_lte_handover  \
0          0.0           occurrence                  12                  12   
1          0.0       total_duration           10.953987            0.199657   
2          0.0     total_packet_num                5477                 100   
3          1.0      packet_loss_num                 385                   0   
4          1.0  packet_loss_rate(%)            7.029396                 0.0   

  after_lte_handover before_SN_addition during_SN_addition after_SN_addition  \
0                 12                  1                  1                 1   
1          11.127297           0.270857           0.020832               1.0   
2               5564                135                 10               500   
3                  0                  0                  0                 0   
4                0.0                0.0                0.0               0.0   

  before_endc_SN_change during_endc_SN_chang